In [ ]:
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import torch
from helper_functions import get_paths_for_en_episodes
from helper_functions import dialogue_json_to_pandas
from tqdm import tqdm

In [ ]:
device = torch.device('cpu')

In [ ]:
# Load DistilBert Classification model from transformer library
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1).to(device=device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load saved best BERT model op CPU (model was trained on GPU in Colab)
model.load_state_dict(torch.load('DistilBERT_best_model.pt', map_location=device))
model.eval()

In [ ]:
# tokenize utterances in dialogue
tokenized_utterances = inference_dialogue.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [ ]:
# pad sentences for inference
max_len = max(map(len,tokenized_utterances))
padded_utterances = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_utterances])
attention_masked_utterances = np.where(padded_utterances != 0,1,0)

In [ ]:
# Convert dialogue into tensors and dataset
X_inference = torch.tensor(padded_utterances)
X_inference_attention = torch.tensor(attention_masked_utterances)

inference_dataset = TensorDataset(X_inference, X_inference_attention)
inference_loader = DataLoader(inference_dataset, batch_size=2, shuffle=False)

In [ ]:
 def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 

In [ ]:
preds = np.zeros([len(inference_dataset), 1])
for i, (x_batch, x_mask) in enumerate(inference_loader):
    print(i)
    outputs = model(x_batch.to(device),attention_mask=x_mask.to(device))

    y_pred = sigmoid(outputs[0].detach().cpu().numpy())

    preds[i*2:(i+1)*2, :] = y_pred


pred_labels = []
for p in preds:
    if p > 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
        
print(pred_labels)